In [88]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from math import radians, cos, sin, asin, sqrt
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import OneHotEncoder


In [89]:
# Load the data
train_data = pd.read_csv('data/X_train.csv')
Y_train = pd.read_csv('data/y_train.csv')
test_data = pd.read_csv('data/X_test.csv')

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18508\3271525685.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('data/X_train.csv')


In [90]:

def clean_construction_time_data(construction_time_series):
    def process_construction_time(x):
        try:
            year = int(x)
            if 1900 <= year <= 2024:  
                return year
            else:
                return np.nan
        except ValueError:
            return np.nan  # Trả về NaN cho các giá trị không phải số hoặc số không hợp lệ

    # Áp dụng hàm xử lý
    cleaned_series = construction_time_series.apply(process_construction_time)
    
    # Tìm giá trị phổ biến nhất (mode) trong các giá trị hợp lệ
    mode_value = cleaned_series.mode().iloc[0]
    
    # Điền các giá trị NaN bằng giá trị phổ biến nhất
    cleaned_series = cleaned_series.fillna(mode_value)
    
    return cleaned_series.astype(int)  # Chuyển đổi tất cả giá trị về kiểu int

# Sử dụng hàm
train_data['constructionTime'] = clean_construction_time_data(train_data['constructionTime'])
test_data['constructionTime'] = clean_construction_time_data(test_data['constructionTime'])
train_data['livingRoom'].replace({'#NAME?': 2}, inplace=True)
test_data['livingRoom'].replace({'#NAME?': 2}, inplace=True)

# We will find distance agnaist each lat and lng from Beijing (lat:39.916668,lon:116.383331)
def distance(lat2, lon2,lat1=39.916668,lon1=116.383331): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2) 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 
    r = 6371
       
    # calculate the result 
    return(c * r) 

train_data['distance'] = train_data.apply(lambda x: distance (x['Lat'],x['Lng']),axis=1)
train_data['building_age'] = 2024 - train_data['constructionTime']

test_data['distance'] = test_data.apply(lambda x: distance (x['Lat'],x['Lng']),axis=1)
test_data['building_age'] = 2024 - test_data['constructionTime']



C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18508\2172149830.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['livingRoom'].replace({'#NAME?': 2}, inplace=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18508\2172149830.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

In [91]:
# in ra giá trị unique của từng cột
train_data['building_age'].unique()

array([ 42,  20,  22,  25,  17,  24,  30,  16,  21,  13,  18,  26,  14,
        35,  19,  32,  39,  37,  15,  29,  34,  27,  11,  36,  43,  45,
        23,  31,  38,  49,  33,  40,  44,  12,  28,  60,  41,   9,  58,
        10,  54,  48,  59,  66,  68,  46,  69,  61,  50,  47,  64,  57,
        51,  70,  56,  67,   8,  62,  53,  52,  65,  71,  63,  74,  90,
        91,  72, 110,  55,  80])

In [92]:
train_data.dtypes

ID                       int64
Lng                    float64
Lat                    float64
tradeTime               object
followers                int64
square                 float64
livingRoom              object
drawingRoom             object
kitchen                  int64
bathRoom                object
floor                   object
buildingType           float64
constructionTime         int32
renovationCondition      int64
buildingStructure        int64
ladderRatio            float64
elevator               float64
fiveYearsProperty      float64
subway                 float64
district                 int64
communityAverage       float64
distance               float64
building_age             int32
dtype: object

In [93]:
for row in range(train_data.shape[1]):
    if(train_data.iloc[:,row].dtype=="O"):
        print("{}: {}\n".format(train_data.columns[row],train_data.iloc[:,row].unique()))

tradeTime: ['2012-11-12' '2014-11-29' '2015-02-05' ... '2010-02-26' '2010-03-05'
 '2010-04-11']

livingRoom: ['2' '1' '3' '4' '5' '6' '9' '7' '0' '8' 2]

drawingRoom: ['1' '2' '0' '3' '4' '中 16' '中 24' '中 14' '底 28' '中 15' '底 11' '5' '低 15'
 '中 22' '中 6' '低 6' '高 14' '底 20' '低 16' '高 12']

bathRoom: ['1' '2' '3' '0' '4' '5' '6' '2006' '未知' '2003' '2005' 1 2 3 0 5 4 2003
 2005 6 2000 1990 2006 1994 7 2004 1996 2011]

floor: ['顶 5' '高 24' '高 12' '低 24' '中 6' '中 14' '底 5' '低 22' '顶 6' '中 9' '低 18'
 '中 17' '低 19' '中 11' '高 23' '高 32' '中 18' '底 6' '低 6' '中 12' '低 7' '低 20'
 '低 14' '中 8' '高 10' '底 7' '高 6' '底 22' '低 28' '底 18' '未知 6' '顶 26' '高 7'
 '中 5' '高 9' '中 16' '高 16' '中 21' '低 21' '低 11' '中 31' '高 18' '高 15'
 '底 28' '低 17' '低 30' '高 27' '底 13' '底 4' '低 12' '高 25' '中 19' '低 16'
 '低 29' '高 22' '顶 4' '中 7' '顶 22' '中 15' '中 22' '底 11' '顶 16' '中 10'
 '中 28' '低 9' '底 10' '高 26' '顶 14' '顶 9' '中 29' '顶 17' '低 26' '高 11'
 '高 30' '低 25' '高 17' '低 10' '顶 7' '低 32' '中 13' '中 25' '底 26' '中 27'
 '顶 

In [94]:
# in ra giá trị unique của từng cột
print(train_data['buildingStructure'].mode().iloc[0])

6


In [95]:
def clean_building_type_data(building_type_series):
    def process_building_type(x):
        if pd.isna(x):
            return 'Unknown'
        elif isinstance(x, (int, float)):
            if x == 1 or x == 1.0:
                return 'Tower'
            elif x == 2 or x == 2.0:
                return 'Bungalow'
            elif x == 3 or x == 3.0:
                return 'Tower and Plate'
            elif x == 4 or x == 4.0:
                return 'Plate'
            elif 0 < x < 1:  # Handling decimal values
                return 'Plate'
            else:
                return 'Plate'
        else:
            return str(x)  # Keep existing string values

    cleaned_series = building_type_series.apply(process_building_type)
    
    return cleaned_series

# Sử dụng hàm
train_data['buildingType'] = clean_building_type_data(train_data['buildingType'])
test_data['buildingType'] = clean_building_type_data(test_data['buildingType'])
def clean_floor_data(floor_series):
    def process_floor(x):
        x = str(x)
        if x == '结构':
            return 6  
        else:
            return x[-2:].strip()  # lấy 2 ký tự cuối và loại bỏ khoảng trắng
    
    cleaned_series = floor_series.apply(process_floor)
    
    # Chuyển đổi thành số nếu có thể
    cleaned_series = pd.to_numeric(cleaned_series, errors='coerce')
    
    # Điền giá trị NaN bằng mode
    median_value = cleaned_series.mode().iloc[0]
    cleaned_series = cleaned_series.fillna(median_value)
    
    return cleaned_series

# Sử dụng hàm
train_data['floor'] = clean_floor_data(train_data['floor'])
test_data['floor'] = clean_floor_data(test_data['floor'])

def clean_bathroom_data(bathroom_series):
    # Chuyển đổi series thành chuỗi
    bathroom_series = bathroom_series.astype(str)
    
    # Hàm để xử lý từng giá trị
    def process_value(x):
        try:
            # Chuyển đổi thành số nguyên
            value = int(float(x))
            # Kiểm tra xem giá trị có nằm trong khoảng hợp lý không (0-6)
            if 0 <= value <= 6:
                return value
            else:
                return np.nan
        except ValueError:
            # Nếu không thể chuyển đổi thành số, trả về NaN
            return np.nan
    
    # Áp dụng hàm xử lý cho series
    cleaned_series = bathroom_series.apply(process_value)
    
    # Điền giá trị thiếu bằng median
    median_value = cleaned_series.mode().iloc[0]
    cleaned_series = cleaned_series.fillna(median_value)
    
    return cleaned_series

# Sử dụng hàm
train_data['bathRoom'] = clean_bathroom_data(train_data['bathRoom'])
test_data['bathRoom'] = clean_bathroom_data(test_data['bathRoom'])

def clean_drawing_room_data(drawing_room_series):
    def process_drawing_room(x):
        x = str(x).lower()
        if x.isdigit():
            return int(x)
        else:
            parts = x.split()
            if len(parts) == 2 and parts[1].isdigit():
                return int(parts[1])
            else:
                return np.nan

    cleaned_series = drawing_room_series.apply(process_drawing_room)
    
    # Điền giá trị NaN bằng median
    median_value = cleaned_series.mode().iloc[0]
    cleaned_series = cleaned_series.fillna(median_value)
    
    return cleaned_series

# Sử dụng hàm
train_data['drawingRoom'] = clean_drawing_room_data(train_data['drawingRoom'])
test_data['drawingRoom'] = clean_drawing_room_data(test_data['drawingRoom'])

In [98]:
train_data.columns

Index(['ID', 'Lng', 'Lat', 'tradeTime', 'followers', 'square', 'livingRoom',
       'drawingRoom', 'kitchen', 'bathRoom', 'floor', 'buildingType',
       'constructionTime', 'renovationCondition', 'buildingStructure',
       'ladderRatio', 'elevator', 'fiveYearsProperty', 'subway', 'district',
       'communityAverage', 'distance', 'building_age'],
      dtype='object')

In [99]:
#lấy những columns cần thiết để training
train = ['Lng', 'Lat', 'tradeTime', 'followers', 'square', 'livingRoom',
       'drawingRoom', 'kitchen', 'bathRoom', 'floor', 'buildingType',
       'constructionTime', 'renovationCondition', 'buildingStructure',
       'ladderRatio', 'elevator', 'fiveYearsProperty', 'subway', 'district',
       'communityAverage', 'distance', 'building_age']
train_data = train_data[train]
test_data = test_data[train]

all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

#obtain all the numerical features
numerical_features = all_features.dtypes[all_features.dtypes != 'object'].index
#apply standardization to each feature
all_features[numerical_features] = all_features[numerical_features].apply( lambda x: (x-x.mean()) / x.std() )
#replace missing values with 0
all_features[numerical_features] = all_features[numerical_features].fillna(0)
#one-hot encoding consider missing values as a category.
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

pd.isna(all_features).any()

Lat                             False
followers                       False
square                          False
drawingRoom                     False
kitchen                         False
                                ...  
buildingType_Plate              False
buildingType_Tower              False
buildingType_Tower and Plate    False
buildingType_Unknown            False
buildingType_nan                False
Length: 2597, dtype: bool

In [100]:
#Tách dữ liệu thành tập huấn luyện và tập kiểm tra
n_train = train_data.shape[0]
train_features = all_features[:n_train]
test_features = all_features[n_train:]
train_labels = Y_train.values

# Tách tập huấn luyện thành tập train và validation
x_train, x_valid,y_train, y_valid = train_test_split(train_features, train_labels, test_size=0.250001, random_state = 27)


In [101]:
rf = RandomForestRegressor(random_state=42,n_estimators=900,max_depth=20,
                                              n_jobs=-1,min_samples_split=10,)

In [102]:
# Fit Model

rf.fit(x_train, y_train)

# Make validation predictions

y_pred = rf.predict(x_valid)

KeyboardInterrupt: 

In [59]:

# Step 1: Predict using RandomForest on test data
rf_test_pred = rf.predict(test_features)
rf_test_pred.shape
target_predictions = rf_test_pred[:, 1]
# Step 4: Create a DataFrame with ID and TARGET
results_df = pd.DataFrame({
    'ID': range(len(rf_test_pred)), 
    'TARGET': target_predictions
})

# Step 5: Export to CSV
results_df.to_csv('predictions1.csv', index=False)

print("Predictions have been saved to 'predictions.csv'")

Predictions have been saved to 'predictions.csv'
